In [ ]:
"""
Evaluation attack models
"""
import cv2
import os
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import foolbox as fb
import tensorflow as tf
import matplotlib.pyplot as plt
import torchvision.datasets as dset
from keras.datasets import cifar10
from torch.utils import data
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import roc_auc_score
from utils.deepsvdd import DeepSVDD_network, pretrain_autoencoder, TrainerDeepSVDD

In [ ]:
# IMPORTANT
# PLEASE CHANGE THE VAULUE OF DeepTaster_forder TO THE DEEPTASTER DIRECTORY 
Home_directory = ''
DeepTaster_forder = Home_directory+"/DeepTaster/"

In [ ]:
# # Specify the cuda setting if you needed
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#load dataset
(X_train, y_train), (X_test1, y_test1) = cifar10.load_data()
y_test1=tf.keras.utils.to_categorical(y_test1)
X_test1=X_test1.reshape(10000,3,32,32)
X_test1=X_test1/255
X_test1=X_test1.astype(np.float32)
random.seed(11)
random_numbers = np.arange(10000)
random.shuffle(random_numbers)
X_test1 = X_test1[random_numbers]
y_test1= y_test1[random_numbers]

In [ ]:
if not os.path.exists( Home_directory + '/DeepTaster/images/attack_models'):
    os.makedirs(Home_directory + '/DeepTaster/images/attack_models')

attack_list = ["DAA","SAA","TLA","MFA","MPA","DATLA","TLPA"]
for attack in attack_list:
    if not os.path.exists( Home_directory + '/DeepTaster/images/attack_models/'+attack):
        os.makedirs(Home_directory + '/DeepTaster/images/attack_models/'+attack)
    for file in os.listdir(Home_directory + '/DeepTaster/models/attack_model/'+attack):
        if file[-2:] !="pt":
            continue
        
        target_model = torch.load(Home_directory + '/DeepTaster/models/attack_model/'+attack+'/'+file,map_location='cpu')
        target_model = target_model.to(device)
        target_model.eval()
        print(file[0:-3])
        bounds = (0,255)
        fmodel = fb.PyTorchModel(target_model, bounds=bounds)
        if not os.path.exists( Home_directory + '/DeepTaster/images/attack_models/'+attack+'/'+file[0:-3]):
            os.makedirs( Home_directory + '/DeepTaster/images/attack_models/'+attack+'/'+file[0:-3])
        for k in range(9):
            X_test=torch.from_numpy(X_test1[0+32*k:32+32*k]).float().to(device)
            X_test=F.interpolate(X_test, size=(224, 224), mode='bilinear', align_corners=False)
            y_test=[]
            for j in range(32):
                y_test.append(torch.argmax(target_model(X_test)[j]))
            y_test=torch.tensor(y_test).to(device)
            adversarial_attack = fb.attacks.FGSM()

            raw, clipped, is_adv = adversarial_attack(fmodel,X_test,y_test, epsilons=0.03)

            for i in range(32):
                plt.figure(num=None, figsize=(2,2), dpi=150)
                plt.imshow((clipped[i]-X_test[i]).cpu().permute(1,2,0));
                plt.axis('off')
                plt.savefig(Home_directory + '/DeepTaster/images/temp'+'/per'+str(i+32*k)+'.jpg', dpi=150,bbox_inches='tight', pad_inches=0)
            for i in range(32):
                img_c1=cv2.imread(os.path.join(Home_directory + '/DeepTaster/images/temp','per'+str(i+32*k)+'.jpg'), 0)
                img_c2 = np.fft.fft2(img_c1)
                img_c3 = np.fft.fftshift(img_c2)
                cv2.imwrite(os.path.join(Home_directory + '/DeepTaster/images/attack_models',attack,file[0:-3]+'/'+str(i+32*k)+'.jpg'),20*np.log(1+np.abs(img_c3)))

In [ ]:
net = DeepSVDD_network().to(device)
state_dict = torch.load(DeepTaster_forder +'models/classifier/pretrained.pth')
c = torch.Tensor(state_dict['center']).to(device)
net=torch.load(DeepTaster_forder +'models/classifier/deepsvdd.pt').to(device)
threshold_file=open(DeepTaster_forder +'models/classifier/threshold.txt','r')
threshold=float(threshold_file.readline())
threshold_file.close()

In [ ]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/DAA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [ ]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [ ]:
print(dataset.class_to_idx)

In [ ]:
print("DAA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Epochs","Model Acc.","Attack Prob.","Copy Detection (%)"))
epochs = ["100","20","40","60","80"]
for i in [1,2,3,4,0]:
    acc_file=open(Home_directory + '/DeepTaster/models/attack_model/DAA/DAA_'+epochs[i]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10', "Stolen", epochs[i],round(acc,2), round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))


In [ ]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/SAA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [ ]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [ ]:
print("SAA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Victim","Suspect", "Ground Truth","Used Dataset","Attack Prob.","Copy Detection (%)"))
data_percentage = ["10", "30","50","70","90"]
for i in range(5):
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10','CIFAR10', "Stolen", data_percentage[i],round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))


In [ ]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/TLA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [ ]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [ ]:

print("TLA results\n")
print('{:<18} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Epochs","Model Acc.","Attack Prob.","Copy Detection (%)"))
data_percentage = ["2","4","6","8"]
for i in range(4):
    acc_file=open(Home_directory + '/DeepTaster/models/attack_model/TLA/Transfer_'+data_percentage[i]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<18} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10 to MNIST', "Stolen", data_percentage[i],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))


In [ ]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/MFA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [ ]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [ ]:
print("MFA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Used Dataset","Model Acc.","Attack Prob.","Copy Detection (%)"))
data_percentage = [ "500","1000","2500"]
for i in range(3):
    acc_file=open(Home_directory + '/DeepTaster/models/attack_model/MFA/Finetuning_'+data_percentage[i]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10', "Stolen", data_percentage[i],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))
for i in range(3,6):
    acc_file=open(Home_directory + '/DeepTaster/models/attack_model/MFA/MNIST_Finetuning_'+data_percentage[i-4]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('MNIST', "Benign", data_percentage[i-3],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] <144 else 0 ))


In [ ]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/MPA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [ ]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [ ]:
print("MPA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Pruned (%)","Model Acc.","Attack Prob.","Copy Detection (%)"))
data_percentage = ["20", "40","60"]
for i in range(3,6):
    acc_file=open(Home_directory + '/DeepTaster/models/attack_model/MPA/Prune_'+data_percentage[i-3]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10', "Stolen", data_percentage[i-3],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i]  >=144 else 0 ))
for i in range(3):
    acc_file=open(Home_directory + '/DeepTaster/models/attack_model/MPA/MNIST_Prune_'+data_percentage[i]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('MNIST', "Benign", data_percentage[i],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] <144 else 0 ))


In [ ]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/DATLA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [ ]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [ ]:
print(dataset.class_to_idx)

In [ ]:
print("DATLA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Epochs","Model Acc.","Attack Prob.","Copy Detection (%)"))
epochs = ["100","20","40","60","80"]
for i in [1,2,3,4,0]:
    acc_file=open(Home_directory + '/DeepTaster/models/attack_model/DATLA/DATLA_'+epochs[i]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10', "Stolen", epochs[i],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))

In [ ]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/TLPA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [ ]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [ ]:
print("TLPA results\n")
print('{:<18} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Used Dataset","Model Acc.","Attack Prob.","Copy Detection (%)"))
epochs = ["10", "30","50","70",'90']
for i in range(5):
    acc_file=open(Home_directory + '/DeepTaster/models/attack_model/TLPA/TLPA_'+epochs[i]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<18} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('MNIST to CIFAR10', "Stolen", epochs [i],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))